# Counterfactual Computation with Addititive Noise

## Causal network

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from pybbn.pptc import PptcModel, create_pptc_model
from pybbn.sampling import sample

np.random.seed(37)

def get_model() -> PptcModel:
    d = nx.DiGraph()
    d.add_nodes_from(['drug', 'gender', 'recovery'])
    d.add_edges_from([('gender', 'drug'), ('gender', 'recovery'), ('drug', 'recovery')])

    p = {
        'gender': {
            'columns': ['gender', '__p__'],
            'data': [
                ['male', 0.51], ['female', 0.49]
            ]
        },
        'drug': {
            'columns': ['gender', 'drug', '__p__'],
            'data': [
                ['female', 'no', 0.24],
                ['female', 'yes', 0.76],
                ['male', 'no', 0.76],
                ['male', 'yes', 0.24]
            ]
        },
        'recovery': {
            'columns': ['gender', 'drug', 'recovery', '__p__'],
            'data': [
                ['female', 'no', 'no', 0.90],
                ['female', 'no', 'yes', 0.10],
                ['female', 'yes', 'no', 0.27],
                ['female', 'yes', 'yes', 0.73],
                ['male', 'no', 'no', 0.99],
                ['male', 'no', 'yes', 0.01],
                ['male', 'yes', 'no', 0.07],
                ['male', 'yes', 'yes', 0.93]
            ]
        }
    }

    m = create_pptc_model(d, p)
    return m

model = get_model()

## Sampling

In [2]:
N = 10_000
Xy = sample(model, max_samples=N) \
    .assign(
        gender=lambda d: d['gender'].map({'male': 1, 'female': 0}),
        drug=lambda d: d['drug'].map({'yes': 1, 'no': 0}),
        recovery=lambda d: d['recovery'].map({'yes': 1, 'no': 0})
    ) \
    .assign(
        U_d=np.random.normal(0, 1, size=N),
        U_r=np.random.normal(0, 1, size=N)
    ) \
    .rename(columns={'gender': 'G', 'drug': 'D', 'recovery': 'R'})

Xy.shape

(10000, 5)

In [3]:
Xy.head()

,G,D,R,U_d,U_r
0,1,0,0,0.104978,0.151996
1,1,1,1,-0.814426,0.988001
2,0,1,0,0.248942,0.548840
3,1,1,1,0.164245,1.118099
4,0,1,1,1.037087,1.884652


## Structural causal models

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

def record(vals, cols):
    return pd.DataFrame([vals], columns=cols)

X, y = Xy[['G', 'D', 'R']], Xy['U_r']
u_model = RandomForestRegressor(random_state=37, n_jobs=-1).fit(X, y)

X, y = Xy[['G', 'D', 'U_r']], Xy['R']
y_model = RandomForestClassifier(random_state=37, n_jobs=-1).fit(X, y)

## Counterfactual 1

We observe a patient who was male, $G=1$, took the drug, $D=1$ and recovered $R=1$. What is the probability of recovery had the patient `not` taken the drug?

In [8]:
G, D, R = 1, 1, 1
_D = 0

In [9]:
u_r = u_model.predict(record([G, D, R], ['G', 'D', 'R']))[0]
u_r

0.009851258822790709

In [10]:
y_model.predict_proba(record([G, _D, u_r], ['G', 'D', 'U_r']))

array([[0.7, 0.3]])

## Counterfactual 2

We observe a patient who was female, $G=0$, took the drug, $D=1$ and recovered $R=1$. What is the probability of recovery had the patient `not` taken the drug?

In [11]:
G, D, R = 0, 1, 1
_D = 0

In [12]:
u_r = u_model.predict(record([G, D, R], ['G', 'D', 'R']))[0]
u_r

-0.043631856154001704

In [13]:
y_model.predict_proba(record([G, _D, u_r], ['G', 'D', 'U_r']))

array([[0.72, 0.28]])